In [1]:
# EuRoC data handler
import pandas as pd
import numpy as np
# class euroc_data():
# 	def __init__(self,groundtruth_path):
# 		gt_frame = pd.read_csv(groundtruth_path,'r')
GROUNDTRUTH_PATH = '/home/cdeng/EuRoC/mav0/\
state_groundtruth_estimate0/data.csv'
IMU_PATH = '/home/cdeng/EuRoC/mav0/imu0/data.csv'

In [2]:
gt_dataframe = pd.read_csv(GROUNDTRUTH_PATH, index_col=0)
imu_dataframe = pd.read_csv(IMU_PATH, index_col=0)
# print(gt_dataframe.head(3))
# print('-----------------')
# print(imu_dataframe.head(3))

In [3]:
# Find the start and end time stamp of the ground truth, 
# and extract relevant imu data frame, gt and imu should be same length  
groudtruth_data = gt_dataframe.iloc[:-1,0:7]
start_time, end_time = groudtruth_data.index[0], groudtruth_data.index[-1]
print (start_time, end_time)

1403636580838555648 1403636762738555392


In [4]:
# Set threshold to 2 000 000 ns, because the time interval between imu data is 5 000 000 ns
threshold = 1000000
[imu_start_idx, imu_end_idx] = [idx for (idx, timestamp) in enumerate(imu_dataframe.index)  \
    if (abs(timestamp-start_time)<threshold or abs(timestamp-end_time)<threshold)]
print (imu_start_idx, imu_end_idx)

216 36596


In [5]:
# for timestamp in imu_dataframe['#timestamp [ns]']:
#     if (abs(timestamp-start_time)<threshold or abs(timestamp-end_time)<threshold):
#         print (timestamp)

In [9]:
# ---------imu data at t-1 is used to predicate gt at t
# Groundtruth is for image 1-n, [first_image_middle_exp_time, last_image_middle_exp_time], that't why the number of 
# datapoints end with 1.
imu_data = imu_dataframe[imu_start_idx-1:imu_end_idx].copy()

if (groudtruth_data.shape[0] != imu_data.shape[0]):
    print("'Error: groundtruth and imu data don't have the same lenght'")
print (imu_data.shape)
print (groudtruth_data.shape)

imu_data.tail(4)
# imu_data.iloc[1:3]
# imu_dataframe[0:6]

(36381, 6)
(36381, 7)


,w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
#timestamp [ns],,,,,,
1403636762718555392,-0.055851,0.184307,-0.169646,8.229414,0.784532,-2.075741
1403636762723555584,-0.108909,0.070511,-0.157080,9.496106,-0.204305,-4.135137
1403636762728555520,-0.077493,0.181514,-0.208043,7.028099,-0.449471,-2.043052
1403636762733555456,-0.086568,0.016755,-0.169646,9.635034,-0.286027,-3.383294


In [12]:
training_input.shape

(25460, 6)

In [51]:
# Use the simple keras deep learning API
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
import time

In [52]:

# After studying the dataset, ignore the last data from ground truth. Because the second last element 
# has the timestamp that happens to be the middle point of image 3661 and 3662 (opened in Office, column index)
datasize = imu_data.shape[0] 
train_val_split_ratio = 0.7
training_size = int(datasize*train_val_split_ratio//10*10)
print('Training size is: {} \n Testing size is: {}'.format(training_size, datasize-training_size))
training_input = imu_data[:training_size]
training_groudtruth = groudtruth_data[:training_size]
testing_input = imu_data[training_size:]
testing_groundtruth = groudtruth_data[training_size:]
# imu_data.iloc[0]
# imu_data.iloc[0]

def find_error(model, batch_size, input_data, groundtruth):
    X = input_data.reshape(input_data.shape[0],1,input_data.shape[1])    
    predictions = model.predict(X,batch_size)
    error = mean_squared_error(predictions,groundtruth)
    return error
def fit_lstm(find_error, log_dir, training_input, training_groudtruth, batch_size, num_epoch, neurons):
    # For now, I set sequence length to 1, in reality, it is better to be 10.
    print (training_input.shape)
    X = training_input.reshape(training_input.shape[0], 1, training_input.shape[1])
    y = training_groudtruth
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2])))
    model.add(Dense(7))
    model.compile(loss='mse', optimizer='adam')
    tbCallBack = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
    for i in range(num_epoch):
        start = time.time()
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0,shuffle=False,stateful=True, callbacks=[tbCallBack])
        end = time.time()
        error = find_error(model,1,training_input, training_groudtruth)
        print ('Error at epoch %d is %f . Time consumed is %f seconds'%(i, error, end-start))
        
        model.reset_states()
    return model

# Find a way to record the error and some other data for visualizing
log_dir = '/home/cdeng/master/tmp/logs/sensor_fusion_LSTM'


training_input = np.array(training_input)[:100,:]
print (training_input.shape)
training_groudtruth = np.array(training_groudtruth)[:100,:]
testing_input = np.array(training_input)
testing_groundtruth = np.array(testing_groundtruth)
lstm_model = fit_lstm(find_error, log_dir, training_input, training_groudtruth, batch_size=1, num_epoch=100,neurons=64)

Training size is: 25460 
 Testing size is: 10921
(100, 6)
(100, 6)
Error at epoch 0 is 0.079088 . Time consumed is 6.086818 seconds
Error at epoch 1 is 0.057904 . Time consumed is 4.647493 seconds
Error at epoch 2 is 0.051958 . Time consumed is 4.624977 seconds
Error at epoch 3 is 0.046044 . Time consumed is 4.930491 seconds
Error at epoch 4 is 0.040157 . Time consumed is 5.032720 seconds
Error at epoch 5 is 0.035281 . Time consumed is 4.675458 seconds
Error at epoch 6 is 0.031209 . Time consumed is 4.630860 seconds
Error at epoch 7 is 0.027284 . Time consumed is 5.005247 seconds
Error at epoch 8 is 0.023677 . Time consumed is 4.605816 seconds
Error at epoch 9 is 0.020567 . Time consumed is 4.611587 seconds
Error at epoch 10 is 0.018099 . Time consumed is 4.729308 seconds
Error at epoch 11 is 0.016002 . Time consumed is 5.162526 seconds
Error at epoch 12 is 0.014265 . Time consumed is 4.608140 seconds
Error at epoch 13 is 0.012847 . Time consumed is 4.687553 seconds
Error at epoch 14 i